In [5]:
import pandas as pd
import numpy as np
import sklearn as sk
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

keras.__version__

'3.11.3'

In [3]:
#importo mi dataframe con dummys
df_encoded = pd.read_csv("qll_limpio_encoded.csv")

In [7]:
df_encoded.head()

,id_lector,id_libro,rating,nombre,nacimiento,titulo,autor,editorial,anio_edicion,isbn,...,pais_republica checa,pais_rumania,pais_samoa americana,pais_san bartolome,pais_san pedro y miquelon,pais_serbia,pais_suiza,pais_turquia,pais_uruguay,pais_venezuela
0,yura,-incluso-el-olvido,6,yura,1996.0,",INCLUSO EL OLVIDO.","VITAS, ROMÁN ROLANDO",DEL CLÉ,2015.0,9789873755132.0,...,0,0,0,0,0,0,0,0,0,0
1,manucasoul,100-anos-de-pulgarcito-la-revista-donde-empezo...,10,manuel,1952.0,100 AÑOS DE PULGARCITO. La revista donde empez...,"GUIRAL, ANTONI",BRUGUERA,2021.0,9788402425171.0,...,0,0,0,0,0,0,0,0,0,0
2,aketza,1001-comics-que-hay-que-leer-antes-de-morir,8,aketza,1981.0,1001 COMICS QUE HAY QUE LEER ANTES DE MORIR,"GRAVETT, PAUL",GRIJALBO,2012.0,9788425347702.0,...,0,0,0,0,0,0,0,0,0,0
3,albertofg1,1080-recetas-de-cocina,9,alberto fernandez,1970.0,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",ALIANZA,2023.0,9788411483582.0,...,0,0,0,0,0,0,0,0,0,0
4,trini,1080-recetas-de-cocina,10,trinidad sanz,1959.0,1080 RECETAS DE COCINA,"ORTEGA, SIMONE",ALIANZA,2023.0,9788411483582.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#quiero que se quede con las columnas numericas y además id_libro e id_lector aunque no sean numéricas
numeric_cols = df_encoded.select_dtypes(include=[np.number]).columns.tolist()
cols_to_add = [col for col in ["id_libro", "id_lector"] if col not in numeric_cols]
cols_to_keep =  cols_to_add + numeric_cols
df_numericas = df_encoded[cols_to_keep].copy()
df_numericas.head()

,id_libro,id_lector,rating,nacimiento,anio_edicion,genero_,genero_-,genero_Hombre,genero_Mujer,genero_literario_,...,pais_republica checa,pais_rumania,pais_samoa americana,pais_san bartolome,pais_san pedro y miquelon,pais_serbia,pais_suiza,pais_turquia,pais_uruguay,pais_venezuela
0,-incluso-el-olvido,yura,6,1996.0,2015.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,100-anos-de-pulgarcito-la-revista-donde-empezo...,manucasoul,10,1952.0,2021.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1001-comics-que-hay-que-leer-antes-de-morir,aketza,8,1981.0,2012.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1080-recetas-de-cocina,albertofg1,9,1970.0,2023.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1080-recetas-de-cocina,trini,10,1959.0,2023.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_emb = df_numericas.copy()

# Convertimos a códigos numéricos consecutivos 
df_emb["id_libro"] = df_emb["id_libro"].astype("category").cat.codes
df_emb["id_lector"] = df_emb["id_lector"].astype("category").cat.codes

n_libros = df_emb["id_libro"].nunique()
n_lectores = df_emb["id_lector"].nunique()

print(f"Libros únicos: {n_libros}, Lectores únicos: {n_lectores}")

Libros únicos: 48548, Lectores únicos: 11289


In [12]:
df_emb.head()

,id_libro,id_lector,rating,nacimiento,anio_edicion,genero_,genero_-,genero_Hombre,genero_Mujer,genero_literario_,...,pais_republica checa,pais_rumania,pais_samoa americana,pais_san bartolome,pais_san pedro y miquelon,pais_serbia,pais_suiza,pais_turquia,pais_uruguay,pais_venezuela
0,0,11224,6,1996.0,2015.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,6,7031,10,1952.0,2021.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23,821,8,1981.0,2012.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,39,879,9,1970.0,2023.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,39,10667,10,1959.0,2023.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:

split_ratio = 0.8
random_seed = 42  
df_emb_shuffled = df_emb.sample(frac=1, random_state=random_seed).reset_index(drop=True)
split_index = int(len(df_emb_shuffled) * split_ratio)
df_train = df_emb_shuffled.iloc[:split_index].copy()
df_test = df_emb_shuffled.iloc[split_index:].copy()
print(f"Train shape: {df_train.shape}, Test shape: {df_test.shape}")

Train shape: (375094, 73), Test shape: (93774, 73)


## Embedding de Libros

In [28]:
n_cats = n_libros
n_embedding_size = min(n_cats // 2 + 1, 50)  # tamaño del vector embedding (máx 50)

model_libro = keras.models.Sequential([
    keras.layers.Input(shape=[1]),
    keras.layers.Embedding(n_cats, n_embedding_size),
    keras.layers.Dense(1)
])

model_libro.compile(loss="mse", optimizer="adam")


history_libro = model_libro.fit(
    df_train["id_libro"], 
    df_train["rating"], 
    epochs=10, 
    batch_size=256, 
    verbose=1
)



Epoch 1/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 40s 27ms/step - loss: 19.2343
Epoch 2/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 35s 24ms/step - loss: 5.1661
Epoch 3/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 36s 25ms/step - loss: 3.5334
Epoch 4/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 36s 24ms/step - loss: 3.0761
Epoch 5/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 36s 25ms/step - loss: 2.9159
Epoch 6/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 39s 27ms/step - loss: 2.8387
Epoch 7/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 36s 25ms/step - loss: 2.7883
Epoch 8/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 37s 26ms/step - loss: 2.7529
Epoch 9/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 36s 25ms/step - loss: 2.7147
Epoch 10/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 33s 23ms/step - loss: 2.6867


In [29]:
# Obtenemos la matriz de pesos del embedding
libro_weights = model_libro.layers[0].get_weights()[0]

# La convertimos en DataFrame
libro_weights = pd.DataFrame(
    libro_weights, 
    columns=[f"libro_emb_{i}" for i in range(n_embedding_size)]
)
libro_weights["id_libro"] = libro_weights.index

libro_weights.head()


,libro_emb_0,libro_emb_1,libro_emb_2,libro_emb_3,libro_emb_4,libro_emb_5,libro_emb_6,libro_emb_7,libro_emb_8,libro_emb_9,...,libro_emb_41,libro_emb_42,libro_emb_43,libro_emb_44,libro_emb_45,libro_emb_46,libro_emb_47,libro_emb_48,libro_emb_49,id_libro
0,-0.085776,-0.088609,0.094594,-0.108803,0.103222,0.108677,-0.073624,-0.047051,0.044028,-0.051352,...,0.123863,-0.081789,-0.052586,0.099882,0.105977,0.069056,-0.053260,-0.125562,0.097704,0
1,-0.076505,-0.091800,0.098876,-0.061638,0.052069,0.127376,-0.090494,-0.074543,0.133547,-0.071598,...,0.089584,-0.089425,-0.053004,0.090876,0.046261,0.061879,-0.136656,-0.058650,0.047163,1
2,-0.198679,-0.129295,0.162138,-0.190701,0.104368,0.150209,-0.178781,-0.120077,0.189927,-0.191428,...,0.145429,-0.114491,-0.172117,0.194959,0.150309,0.144179,-0.161780,-0.135247,0.201560,2
3,-0.133832,-0.174189,0.206259,-0.163049,0.207703,0.136512,-0.205572,-0.159525,0.147944,-0.169346,...,0.218904,-0.182330,-0.213583,0.168870,0.218046,0.174053,-0.221556,-0.162370,0.133412,3
4,0.006934,-0.043501,-0.028433,0.021945,0.051694,-0.036411,0.004506,0.017129,0.036090,-0.025014,...,0.022713,0.031621,-0.029503,0.030277,0.015662,-0.036399,0.023944,0.019082,-0.020932,4


In [30]:
df_train_libro = df_train.merge(libro_weights, on="id_libro").drop(columns=["id_libro"])
df_test_libro = df_test.merge(libro_weights, on="id_libro").drop(columns=["id_libro"])

In [31]:
from sklearn.ensemble import RandomForestRegressor

X_train = df_train_libro[df_train_libro.columns.drop("rating")]
y_train = df_train_libro["rating"]
X_test = df_test_libro[df_test_libro.columns.drop("rating")]
y_test = df_test_libro["rating"]



rf = RandomForestRegressor(
    n_estimators=100,  
    max_depth=5,     
    n_jobs=-1,         
    random_state=42
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(sk.metrics.root_mean_squared_error(y_test, y_pred))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
y_pred_train = rf.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"RMSE train: {rmse_train:.4f}")
print(f"RMSE test: {rmse:.4f}")


1.8905216934110924
RMSE train: 1.6066
RMSE test: 1.8905


## Embedding de Lectores

In [32]:

n_cats = n_lectores
n_embedding_size = min(n_cats // 2 + 1, 50)

model_lector = keras.models.Sequential([
    keras.layers.Input(shape=[1]),
    keras.layers.Embedding(n_cats, n_embedding_size),
    keras.layers.Dense(1)
])

model_lector.compile(loss="mse", optimizer="adam")

history_lector = model_lector.fit(
    df_train["id_lector"], 
    df_train["rating"], 
    epochs=10, 
    batch_size=256, 
    verbose=1
)

lector_weights = model_lector.layers[0].get_weights()[0]
lector_weights = pd.DataFrame(
    lector_weights, 
    columns=[f"lector_emb_{i}" for i in range(n_embedding_size)]
)
lector_weights["id_lector"] = lector_weights.index


Epoch 1/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 14.6107
Epoch 2/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 3.6054
Epoch 3/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 3.1442
Epoch 4/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 3.0111
Epoch 5/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 2.9464
Epoch 6/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 2.9112
Epoch 7/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 2.8878
Epoch 8/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 2.8692
Epoch 9/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 2.8541
Epoch 10/10
1466/1466 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 2.8451


In [37]:
df_train_lectores = df_train.merge(lector_weights, on="id_lector").drop(columns=["id_lector"])
df_test_lectores = df_test.merge(lector_weights, on="id_lector").drop(columns=["id_lector"])

In [38]:
from sklearn.ensemble import RandomForestRegressor

X_train = df_train_lectores[df_train_lectores.columns.drop("rating")]
y_train = df_train_lectores["rating"]
X_test = df_test_lectores[df_test_lectores.columns.drop("rating")]
y_test = df_test_lectores["rating"]



rf = RandomForestRegressor(
    n_estimators=100,  
    max_depth=5,     
    n_jobs=-1,         
    random_state=42
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(sk.metrics.root_mean_squared_error(y_test, y_pred))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
y_pred_train = rf.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"RMSE train: {rmse_train:.4f}")
print(f"RMSE test: {rmse:.4f}")

1.7687775748600696
RMSE train: 1.6753
RMSE test: 1.7688


## Embedding de libro y lector

In [39]:
# Volvemos a unir con los embeddings
df_emb_final = (
    df_emb
    .merge(libro_weights, on="id_libro", how="left")
    .merge(lector_weights, on="id_lector", how="left")
)

print(df_emb_final.shape)
df_emb_final.head()


(468868, 173)


,id_libro,id_lector,rating,nacimiento,anio_edicion,genero_,genero_-,genero_Hombre,genero_Mujer,genero_literario_,...,lector_emb_40,lector_emb_41,lector_emb_42,lector_emb_43,lector_emb_44,lector_emb_45,lector_emb_46,lector_emb_47,lector_emb_48,lector_emb_49
0,0,11224,6,1996.0,2015.0,0,0,0,1,0,...,-0.180548,-0.166767,0.188284,-0.177448,-0.151772,-0.186685,0.107004,-0.142230,0.140916,-0.190540
1,6,7031,10,1952.0,2021.0,0,0,1,0,0,...,-0.102105,-0.153808,0.104321,-0.184817,-0.141968,-0.164170,0.111896,-0.159906,0.210957,-0.131810
2,23,821,8,1981.0,2012.0,0,0,1,0,0,...,-0.086489,-0.113321,0.108224,-0.198176,-0.129121,-0.158196,0.117312,-0.111737,0.176625,-0.120953
3,39,879,9,1970.0,2023.0,0,0,1,0,0,...,-0.134010,-0.152151,0.121895,-0.145019,-0.205322,-0.193862,0.165241,-0.133884,0.173387,-0.146655
4,39,10667,10,1959.0,2023.0,0,1,0,0,0,...,-0.147524,-0.099817,0.097632,-0.131543,-0.130904,-0.149633,0.083955,-0.109185,0.166308,-0.144748


In [40]:
split_ratio = 0.8
random_seed = 42  
df_emb_shuffled = df_emb_final.sample(frac=1, random_state=random_seed).reset_index(drop=True)
split_index = int(len(df_emb_shuffled) * split_ratio)
df_train = df_emb_shuffled.iloc[:split_index].copy()
df_test = df_emb_shuffled.iloc[split_index:].copy()
print(f"Train shape: {df_train.shape}, Test shape: {df_test.shape}")

Train shape: (375094, 173), Test shape: (93774, 173)


In [41]:
X_train = df_train[df_train.columns.drop("rating")]
y_train = df_train["rating"]
X_test = df_test[df_test.columns.drop("rating")]
y_test = df_test["rating"]

rf = RandomForestRegressor(
    n_estimators=100,  
    max_depth=5,     
    n_jobs=-1,         
    random_state=42
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(sk.metrics.root_mean_squared_error(y_test, y_pred))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
y_pred_train = rf.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"RMSE train: {rmse_train:.4f}")
print(f"RMSE test: {rmse:.4f}")

1.840279967690994
RMSE train: 1.5564
RMSE test: 1.8403
